<H1>FBXL4</h1>
<p>Todo-short intro about gene and associated diseases</p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import pyphetools
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import ContentValidator


print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.7.4


In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199")
metadata.default_versions_with_hpo(version=hpo_version)

In [3]:
df = pd.read_excel('input/FBXL4-data.xlsx')

<h1>Data ingest strategy</h1>
<p>We use classes of pyphetool to encode columns that contain the disease identifier, the age, sex, and phenotypes.
For now, we add a column with an identifier (which is required) - we should get this from the original publications - and also
add the PubMed identifiers (instead of DOI), for consistency with the other phenopackets in this project.</p>

In [4]:
df.head(2)

,omim_id,omim_title,omim_name,hgnc_id,gene_symbol,Zigosity,Location,Variant annotation,Consequence,Refseq,...,Protein structure,Age of diagnosis,Gender,Age at death,Phenotype,Prenatal ultrasound phenotype,MRI phenotype,Cardiac phenotype,Family history,Source
0,615471,MITOCHONDRIAL DNA DEPLETION SYNDROME 13 (ENCEPHALOMYOPATHIC TYPE),Mitochondrial DNA depletion syndrome 13 (encephalomyopathic type),HGNC:13601,FBXL4,Compound heterozygous,NM_001278716.1/NM_001278716.1,c.772G>T(p.Gly258*)/c.1061G>C(p.Trp354Ser),nonsense/missense,NM_001278716.1(FBXL4):c.772G>T;(p.Gly258*)/NM_001278716.1(FBXL4):c.1060T>C;(p.Trp354Arg),...,NaN,Fetus,NaN,NaN,"abnormal eyebrow morphology HP:0000534 , triangular face HP:0000325, abnormal shape of the palpebral fissure HP:0200005, mandibular prognathia HP:0000303, increased serum lactate HP:0002151 HP:0002151, increased serum pyruvate HP:0003542",NaN,NaN,NaN,"Primigravid, no family history",https://doi.org/10.24953/turkjped.2021.05.025
1,615471,MITOCHONDRIAL DNA DEPLETION SYNDROME 13 (ENCEPHALOMYOPATHIC TYPE),Mitochondrial DNA depletion syndrome 13 (encephalomyopathic type),HGNC:13601,FBXL4,homozygous,NM_001278716.1,c.993dupA (p.L332Tfs*3),NaN,NM_001278716.1(FBXL4):c.993dup,...,truncated protein. Leucine position 332 changed to arginine and termination,1.5,NaN,NaN,"global developmental delay HP:0001263, hypotonia HP:0001252, abnormal eating behavior HP:0100738, hypertelorism HP:0000316, short palpebral fissure HP:0012745, internuclear ophthalmoplegia HP:0030773, wide nasal bridge HP:0000431",NaN,abnormality of brain morphology HP:0012443,NaN,no family history,https://doi.org/10.24953/turkjped.2021.05.025


In [5]:
# We need to provide an id for a valid phenopacket. TODO - get original IDs from publications if possible

import random
import string

def get_random_string(length):
    letters = string.ascii_lowercase
    result_str = ''.join(random.choice(letters) for i in range(length))
    return result_str
    
df['patient_id'] = df['omim_id'].apply(lambda x: get_random_string(10))
df.set_index('patient_id', inplace=True)

<h3>PMID and title</h3>
<p>For consistency with the rest of the phenopackets, we like to have PMID and title. The following adds new columns to the table starting from the DOI, but it would be nicer to have the PMID info in the original table</p>

In [6]:
source_d = {'https://doi.org/10.24953/turkjped.2021.05.025': ["PMID:34738379", "Entero-encephalopathy due to FBXL4-related mtDNA depletion syndrome"],
    'https://doi.org/10.3389/fgene.2019.01300':["PMID:31969900"," Molecular Characterization of New FBXL4 Mutations in Patients With mtDNA Depletion Syndrome"],
    'https://doi.org/10.24953/turkjped.2020.04.016' : ['PMID:32779419','Different clinical presentation in a patient with two novel pathogenic variants of the FBXL4 gene'],
    'https://doi.org/10.1111/cge.12894':["PMID:27743463","FBXL4 defects are common in patients with congenital lactic acidemia and encephalomyopathic mitochondrial DNA depletion syndrome"],       
    'doi: 10.1007/8904_2015_491':["PMID:26404457","Detailed Biochemical and Bioenergetic Characterization of FBXL4-Related Encephalomyopathic Mitochondrial DNA Depletion"],
    'https://doi.org/10.3389/fgene.2019.00039': ["PMID:30804983", "FBXL4-Related Mitochondrial DNA Depletion Syndrome 13 (MTDPS13): A Case Report With a Comprehensive Mutation Review"],
    'https://doi.org/10.1159/000515928':["PMID:34602956","A Mild Phenotype of Mitochondrial DNA Depletion Syndrome Type 13 with a Novel FBXL4 Variant"],
    'https://doi.org/10.1016/j.gene.2019.01.049':["PMID:30771478", "Whole exome sequencing revealed mutations in FBXL4, UNC80, and ADK in Thai patients with severe intellectual disabilities"],
    'https://doi.org/10.1016/j.ajhg.2013.07.016':["PMID:23993194", "Mutations in FBXL4, encoding a mitochondrial protein, cause early-onset mitochondrial encephalomyopathy"],
}


df['PMID'] = df['Source'].apply(lambda x: source_d.get(x)[0])
df['title']= df['Source'].apply(lambda x: source_d.get(x)[1])
df[['PMID', 'title']].head()

,PMID,title
patient_id,,
kczfusqwrq,PMID:34738379,Entero-encephalopathy due to FBXL4-related mtDNA depletion syndrome
ssiihhjuif,PMID:34738379,Entero-encephalopathy due to FBXL4-related mtDNA depletion syndrome
cuihutsekf,PMID:34738379,Entero-encephalopathy due to FBXL4-related mtDNA depletion syndrome
usdfgrqooc,PMID:34738379,Entero-encephalopathy due to FBXL4-related mtDNA depletion syndrome
mislspnpks,PMID:31969900,Molecular Characterization of New FBXL4 Mutations in Patients With mtDNA Depletion Syndrome


In [7]:
df.columns

Index(['omim_id', 'omim_title', 'omim_name', 'hgnc_id', 'gene_symbol',
       'Zigosity', 'Location', 'Variant annotation', 'Consequence', 'Refseq',
       'Protein ID', 'Genomic location', 'ACGM classification',
       'Protein structure', 'Age of diagnosis', 'Gender', 'Age at death',
       'Phenotype', 'Prenatal ultrasound phenotype', 'MRI phenotype',
       'Cardiac phenotype', 'Family history', 'Source', 'PMID', 'title'],
      dtype='object')

<h1>Variants</h1>
<p>Variant data is available in strings such as c.772G>T(p.Gly258*)/c.1061G>C(p.Trp354Ser) or c.772G>T(p.Gly258*) for homozygous variants.
We need to extract the HGVS cDNA string to pass to Variant Validator</p>

In [8]:
def extract_cdna(variant):
    """
    split strings like c.772G>T(p.Gly258*) on the open-parenthesis symbol and return the first part
    """
    return variant.split("(")[0]
    
def extract_variant_1(variants):
    """
    Split on the slash ("/") and return the first part (or entire string for homozygous)
    """
    v1 = variants.split("/")[0]
    return extract_cdna(v1)

def extract_variant_2(variants):
    """
    Split on the slash ("/") and return the second part (or entire string for homozygous)
    """
    fields = variants.split("/")
    if len(fields) == 2:
        return extract_cdna(fields[1])
    else:
        # there was only one variant
        return extract_cdna(variants)

In [9]:
df["var1"] = df['Variant annotation'].apply(lambda x: extract_variant_1(x))
df["var2"] = df['Variant annotation'].apply(lambda x: extract_variant_2(x))

In [10]:
var1_list = df["var1"].unique()
var2_list = df["var2"].unique()
var_set = set()
var_set.update(var1_list)
var_set.update(var2_list)
variant_d = {}
hg38 = "hg38"
FBXL4_transcript = "NM_001278716.1"
vvalidator = VariantValidator(genome_build=hg38, transcript=FBXL4_transcript)
for v in var_set:
    var = vvalidator.encode_hgvs(v)
    variant_d[v] = var
    print(f"{v} - {var}")

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_001278716.1%3Ac.1652T>A/NM_001278716.1?content-type=application%2Fjson
c.1652T>A - chr6:98875465A>T
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_001278716.1%3Ac.1641_1642delTG/NM_001278716.1?content-type=application%2Fjson
c.1641_1642delTG - chr6:98875474TCA>T
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_001278716.1%3Ac.1703G>C/NM_001278716.1?content-type=application%2Fjson
c.1703G>C - chr6:98874441C>G
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_001278716.1%3Ac.106A>T/NM_001278716.1?content-type=application%2Fjson
c.106A>T - chr6:98926883T>A
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_001278716.1%3Ac.614T>C/NM_001278716.1?content-type=application%2Fjson
c.614T>C - chr6:98917618A>G
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_001278716.1%3Ac.1442T > C /NM_00

<h2>Mapping the age column</h2>
<p>We add a dictionary to the corresponding ISO 8601 age strings. Anything not here will be mapped to "not provided"</p>

In [11]:
string_to_iso_dict = {
    "1.5": "P1Y6M",
    "3.5": "P3Y6M",
    'birth': "P1D",
    '51 days': "P1M21D",
    '19 months': "P1Y7M",
    '10 monhts': "P10M",
    '14 months': "P1Y2M",
    '23 months': "P1Y11M",
    '22 mohts': "P1Y10M",
    '9 months': "P9M",
    '12 years': "P12Y",
    '1 day of age': "P1D",
    '4.5 years': "P4Y6M",
    '4 months': "P4M",
    '1 month': "P1M"
}
ageMapper = AgeColumnMapper.custom_dictionary(column_name='Age of diagnosis', string_to_iso_d=string_to_iso_dict)
ageMapper.preview_column(df['Age of diagnosis']).head()

,original column contents,age
0,Fetus,NOT_PROVIDED
1,1.5,P1Y6M
2,3.5,P3Y6M
3,NaN,NOT_PROVIDED
4,birth,P1D


In [12]:
sexMapper = SexColumnMapper(male_symbol="M", female_symbol="F", unknown_symbol="nan", column_name="Gender")
sexMapper.preview_column(df['Gender']).head()

,original column contents,sex
0,NaN,UNKNOWN
1,NaN,UNKNOWN
2,NaN,UNKNOWN
3,F,FEMALE
4,F,FEMALE


In [13]:
# todo, add vital status to encoder or encode as HPO terms
df['Age at death']

patient_id
kczfusqwrq                       NaN
ssiihhjuif                       NaN
cuihutsekf                       NaN
usdfgrqooc                  7 months
mislspnpks                 10 months
ihifupjyte                    4 days
njgzypocxt           11 months alive
oevrntfcwz           19 months alive
szhdfbdgel           10 months alive
clcxstntzz           14 months alive
bhpgkbnkiw                    5 days
opyjinzmdr           23 months alive
gquoftluyd           22 months alive
zsdipxdimw                       NaN
zjerbrqznk                    3 days
ifvjtukdqs                  9 monhts
hxffpqppaf                  12 years
mgvwsvarth           14 months alive
rctbkzttla    2 years 8 months alive
usgxuzfmie    4 years 6 months alive
dckstjovsl                   3 years
hvubnxsgdb                       NaN
bghtzfxdnv                   4 years
pqpuxurlwu                       NaN
wpujzzjmru                       NaN
ejdxsrfalb                       NaN
hgavzkcuhe                 

<h1>Mapping phenotypic features</h1>
<p>We use the CustomColumnMapper class, which uses text mining to recognize one or more HPO terms in each table cell. The preview
column function can be used to check results, which appear very accurate. Each mapper is put into a map that is used to encode the entire cohort, below</p>

In [14]:
mapper_d = {}

In [15]:
phenotypeColumnMapper = CustomColumnMapper(concept_recognizer=hpo_cr)
phenotypeColumnMapper.preview_column(df['Phenotype'])
mapper_d['Phenotype'] = phenotypeColumnMapper

In [16]:
prenatalUSmapper =  CustomColumnMapper(concept_recognizer=hpo_cr)
prenatalUSmapper.preview_column(df['Prenatal ultrasound phenotype'])
mapper_d['Prenatal ultrasound phenotype'] = prenatalUSmapper

In [17]:
mriMapper = CustomColumnMapper(concept_recognizer=hpo_cr)
mriMapper.preview_column(df['MRI phenotype'])
mapper_d['MRI phenotype'] = mriMapper

In [18]:
cardiacMapper = CustomColumnMapper(concept_recognizer=hpo_cr)
cardiacMapper.preview_column(df['Cardiac phenotype'])
mapper_d['Cardiac phenotype'] = cardiacMapper

<h1>Putting it all together</h1>

In [19]:
mdds13 = Disease(disease_id="OMIM:615471", disease_label="Mitochondrial DNA depletion syndrome 13 (encephalomyopathic type)")
disease_d = {"615471": mdds13}
diseaseMapper = DiseaseIdColumnMapper(column_name="omim_id", disease_id_map=disease_d)

In [20]:
encoder = MixedCohortEncoder(df=df,
                            hpo_cr=hpo_cr,
                             column_mapper_d=mapper_d,
                             individual_column_name="patient_id",
                             disease_id_mapper=diseaseMapper,
                             pmid_column="PMID",
                             title_column="title",
                             sexmapper=sexMapper,
                             agemapper=ageMapper,
                             metadata=metadata
                        )

In [21]:
individuals = encoder.get_individuals()

In [23]:
# retrieve the variant strings and add Variant objects to each individual
# the individual id (i.id) is also the index of the pandas dataframe
for i in individuals:
    print(i.id)
    row = df.loc[i.id] 
    #print(row)
    v1 = row['var1']
    v2 = row['var2']
    print(f"v1={v1} and v2={v2}")
    if v1 == v2:
        var1 = variant_d.get(v1)
        var1.set_homozygous()
        i.add_variant(var1)
    else:
        var1 = variant_d.get(v1)
        var2 = variant_d.get(v2)
        var1.set_heterozygous()
        var2.set_heterozygous()
        i.add_variant(var1)
        i.add_variant(var2)
    

kczfusqwrq
v1=c.772G>T and v2=c.1061G>C
ssiihhjuif
v1=c.993dupA  and v2=c.993dupA 
cuihutsekf
v1=c.1442T > C  and v2=c.1442T > C 
usdfgrqooc
v1=c.1750T>C and v2=c.1750T>C
mislspnpks
v1=c.858+5G>C and v2=c.1510T>C 
ihifupjyte
v1=c.851_852del and v2=c.851_852del
njgzypocxt
v1=c.772G>T and v2=c.1061G>C
oevrntfcwz
v1=c.219T>A and v2=c.219T>A
szhdfbdgel
v1=c.1641_1642del and v2=c.1641_1642del
clcxstntzz
v1=c.1641_1642del and v2=c.1641_1642del
bhpgkbnkiw
v1=c.1687C>T and v2=c.1622C>T
opyjinzmdr
v1=c.1389+3_1389+6delAAGT and v2=c.1838T>A
gquoftluyd
v1=c.64C>T and v2=c.1444C>T
zsdipxdimw
v1=c.1703G>C and v2=c.1703G>C
zjerbrqznk
v1=c.273_277del and v2=c.273_277del
ifvjtukdqs
v1=c.1641_1642delTG and v2=c.1232G>A
hxffpqppaf
v1=c.64C>T and v2=c.1641_1642delTG
mgvwsvarth
v1=c.1641_1642delTG and v2=c.1641_1642delTG
rctbkzttla
v1=c.1303C>T  and v2=c.1303C>T 
usgxuzfmie
v1=c.486T>G  and v2=c.486T>G 
dckstjovsl
v1=c.1303C>T  and v2=c.1303C>T 
hvubnxsgdb
v1=c.1703G>C and v2=c.1703G>C
bghtzfxdnv
v1=c.144

<h1>Validation</h1>
<p>pyphetools offers a quick validation that phenopackets contain a mininum number of variants and HPO terms.
We recommend additional validation with <a href="https://github.com/phenopackets/phenopacket-tools">phenopacket-tools</a>.</p>

In [24]:
cvalidator = ContentValidator(min_var=1, min_hpo=1)
errors = cvalidator.validate_phenopacket_list([i.to_ga4gh_phenopacket(metadata.to_ga4gh()) for i in individuals])
print(f"We found {len(errors)} validation errors")

We found 0 validation errors


<h2>Visualization</h2>

In [25]:
from IPython.display import HTML, display
phenopackets = [i.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh()) for i in individuals]
table = PhenopacketTable(phenopacket_list=phenopackets)
display(HTML(table.to_html()))